#Mining the Social Web, 2nd Edition

##Appendix B: OAuth Primer

This IPython Notebook provides an interactive way to follow along with and explore the numbered examples from [_Mining the Social Web (2nd Edition)_](http://bit.ly/135dHfs). The intent behind this notebook is to reinforce the concepts from the sample code in a fun, convenient, and effective way. This notebook assumes that you are reading along with the book and have the context of the discussion as you work through these exercises.

In the somewhat unlikely event that you've somehow stumbled across this notebook outside of its context on GitHub, [you can find the full source code repository here](http://bit.ly/16kGNyb).

## Copyright and Licensing

You are free to use or adapt this notebook for any purpose you'd like. However, please respect the [Simplified BSD License](https://github.com/ptwobrussell/Mining-the-Social-Web-2nd-Edition/blob/master/LICENSE.txt) that governs its use.

## Notes

While the chapters in the book opt to simplify the discussion by avoiding a discussion of OAuth and instead opting to use application credentials provided by social web properties for API access, this notebook demonstrates how to implement some OAuth flows for several of the more prominent social web properties. While IPython Notebook is used for consistency and ease of learning, and in some cases, this actually adds a little bit of extra complexity in some cases given the nature of embedding a web server and handling asynchronous callbacks. (Still, the overall code should be straightforward to adapt as needed.)

# Twitter OAuth 1.0a Flow with IPython Notebook

Twitter implements OAuth 1.0A as its standard authentication mechanism, and in order to use it to make requests to Twitter's API, you'll need to go to https://dev.twitter.com/apps and create a sample application. There are three items you'll need to note for an OAuth 1.0A workflow, a consumer key and consumer secret that identify the application as well as the oauth_callback URL that tells Twitter where redirect back to after the user has authorized the application. Note that you will need an ordinary Twitter account in order to login, create an app, and get these credentials. Keep in mind that for development purposes or for accessing your own account's data, you can simply use the oauth token and oauth token secret that are provided in your appliation settings to authenticate as opposed to going through the steps here. The process of obtaining an the oauth token and oauth token secret is fairly straight forward (especially with the help of a good library), but an implementation in IPython Notebook is a bit tricker due to the nature of embedding a web server, capturing information within web server contexts, and handling the various redirects along the way.

You must ensure that your browser is not blocking popups in order for this script to work.

<img src="files/resources/ch01-twitter/images/Twitter-AppCredentials-oauth_callback.png" width="600px">

## Example 1. Twitter OAuth 1.0a Flow

In [ ]:
import json
from flask import Flask, request
from threading import Timer
from IPython.display import IFrame
from IPython.display import display
from IPython.display import Javascript as JS

import twitter
from twitter.oauth_dance import parse_oauth_tokens
from twitter.oauth import read_token_file, write_token_file

OAUTH_FILE = "/tmp/twitter_oauth"

# XXX: Go to http://twitter.com/apps/new to create an app and get values
# for these credentials that you'll need to provide in place of these
# empty string values that are defined as placeholders.
# See https://dev.twitter.com/docs/auth/oauth for more information 
# on Twitter's OAuth implementation and ensure that *oauth_callback*
# is defined in your application settings as shown below if you are 
# using Flask in this IPython Notebook

# Define a few variables that will bleed into the lexical scope of a couple of 
# functions below
CONSUMER_KEY = ''
CONSUMER_SECRET = ''
oauth_callback = 'http://127.0.0.1:5000/oauth_helper'
    
# Setup a callback handler for when Twitter redirects back to us after the user authorizes the app

webserver = Flask("TwitterOAuth")
@webserver.route("/oauth_helper")
def oauth_helper():
    
    oauth_verifier = request.args.get('oauth_verifier')

    # Pick back up credentials from ipynb_oauth_dance
    oauth_token, oauth_token_secret = read_token_file(OAUTH_FILE)
    
    _twitter = twitter.Twitter(
        auth=twitter.OAuth(
            oauth_token, oauth_token_secret, CONSUMER_KEY, CONSUMER_SECRET),
        format='', api_version=None)

    oauth_token, oauth_token_secret = parse_oauth_tokens(
        _twitter.oauth.access_token(oauth_verifier=oauth_verifier))

    # This web server only needs to service one request, so shut it down
    shutdown_after_request = request.environ.get('werkzeug.server.shutdown')
    shutdown_after_request()

    # Write out the final credentials that can be picked up after the blocking
    # call to webserver.run() below.
    write_token_file(OAUTH_FILE, oauth_token, oauth_token_secret)
    return "%s %s written to %s" % (oauth_token, oauth_token_secret, OAUTH_FILE)


# To handle Twitter's OAuth 1.0a implementation, we'll just need to implement a custom
# "oauth dance" and will closely follower the pattern defined in twitter.oauth_dance.

def ipynb_oauth_dance():
    
    _twitter = twitter.Twitter(
        auth=twitter.OAuth('', '', CONSUMER_KEY, CONSUMER_SECRET),
        format='', api_version=None)

    oauth_token, oauth_token_secret = parse_oauth_tokens(
            _twitter.oauth.request_token(oauth_callback=oauth_callback))

    # Need to write these interim values out to a file to pick up on the callback from Twitter
    # that is handled by the web server in /oauth_helper
    write_token_file(OAUTH_FILE, oauth_token, oauth_token_secret)
    
    oauth_url = ('http://api.twitter.com/oauth/authorize?oauth_token=' + oauth_token)
    
    # Tap the browser's native capabilities to access the web server through a new window to get
    # user authorization
    display(JS("window.open('%s')" % oauth_url))


# After the webserver.run() blocking call, start the oauth dance that will ultimately
# cause Twitter to redirect a request back to it. Once that request is serviced, the web
# server will shutdown, and program flow will resume with the OAUTH_FILE containing the
# necessary credentials
Timer(1, lambda: ipynb_oauth_dance()).start()

webserver.run(host='0.0.0.0')

# The values that are read from this file are written out at
# the end of /oauth_helper
oauth_token, oauth_token_secret = read_token_file(OAUTH_FILE)

# These 4 credentials are what is needed to authorize the application
auth = twitter.oauth.OAuth(oauth_token, oauth_token_secret,
                               CONSUMER_KEY, CONSUMER_SECRET)
    
twitter_api = twitter.Twitter(auth=auth)

print(twitter_api)

# Facebook OAuth 2.0 Flow with IPython Notebook

Facebook implements OAuth 2.0 as its standard authentication mechanism, and this example demonstrates how get an access token for making API requests once you've created an app and gotten a "client id" value that can be used to initiate an OAuth flow. Note that you will need an ordinary Facebook account in order to login, create an app, and get these credentials. You can create an app through the "Developer" section of your account settings as shown below or by navigating directly to https://developers.facebook.com/apps/. During development or debugging cycles, or to just access data in your own account, you may sometimes find it convenient to also reference the access token that's available to you through the Graph API Explorer tool at https://developers.facebook.com/tools/explorer as opposed to using the flow described here. The process of obtaining an access token is fairly straight forward, but an implementation in IPython Notebook is a bit tricker due to the nature of embedding a web server, capturing information within web server contexts, and handling the various redirects along the way.

You must ensure that your browser is not blocking popups in order for this script to work.
<br />
<br />
<img src="files/resources/ch02-facebook/images/FB_create_app2.png" width="400px"><br />
Create apps at https://developers.facebook.com/apps/<br />
<br />
<br />
<img src="files/resources/ch02-facebook/images/FB_create_app1.png" width="400px"><br />
Clicking on the "Edit" button provides detailed information<br />
about the app, including an opportunity to set its "Site URL"<br />
value that you'll need for the OAuth flow (described later in<br />
this notebook.)

## Example 2. Facebook OAuth 2.0 Flow

In [ ]:
import urllib
from flask import Flask, request
from threading import Timer
from IPython.display import display
from IPython.display import Javascript as JS

# XXX: Get this value from your Facebook application's settings for the OAuth flow
# at https://developers.facebook.com/apps

APP_ID = '' 

# This value is where Facebook will redirect. We'll configure an embedded
# web server to be serving requests here

REDIRECT_URI = 'http://localhost:5000/oauth_helper'

# You could customize which extended permissions are being requested for your app
# by adding additional items to the list below. See
# https://developers.facebook.com/docs/reference/login/extended-permissions/

EXTENDED_PERMS = ['user_likes', 'friends_likes']

# A temporary file to store a code from the web server

OAUTH_FILE = 'resources/ch02-facebook/access_token.txt'

# Configure an emedded web server that accepts one request, parses
# the fragment identifier out of the browser window redirects to another
# handler with the parsed out value in the query string where it can be captured
# and stored to disk. (A webserver cannot capture information in the fragment 
# identifier or that work would simply be done in here.)

webserver = Flask("FacebookOAuth")
@webserver.route("/oauth_helper")
def oauth_helper():
    return '''<script type="text/javascript">
    var at = window.location.hash.substring("access_token=".length+1).split("&")[0]; 
    setTimeout(function() { window.location = "/access_token_capture?access_token=" + at }, 1000 /*ms*/);
    </script>'''

# Parses out a query string parameter and stores it to disk. This is required because
# the address space that Flask uses is not shared with IPython Notebook, so there is really
# no other way to share the information than to store it to a file and access it afterward
@webserver.route("/access_token_capture")
def access_token_capture():
    access_token = request.args.get('access_token')
    f = open(OAUTH_FILE, 'w') # Store the code as a file
    f.write(access_token)
    f.close()
    
    # It is safe (and convenient) to shut down the web server after this request
    shutdown_after_request = request.environ.get('werkzeug.server.shutdown')
    shutdown_after_request()
    return access_token


# Send an OAuth request to Facebook, handle the redirect, and display the access
# token that's included in the redirect for the user to copy and paste
    
args = dict(client_id=APP_ID, redirect_uri=REDIRECT_URI,
            scope=','.join(EXTENDED_PERMS), type='user_agent', display='popup'
            )

oauth_url = 'https://facebook.com/dialog/oauth?' + urllib.parse.urlencode(args)

Timer(1, lambda: display(JS("window.open('%s')" % oauth_url))).start()


webserver.run(host='0.0.0.0')

access_token = open(OAUTH_FILE).read()

print(access_token)

# LinkedIn OAuth 2.0 Flow with IPython Notebook
LinkedIn implements OAuth 2.0 as one of its standard authentication mechanism, and "Example 3" demonstrates how to use it to get an access token for making API requests once you've created an app and gotten the "API Key" and "Secret Key" values that are part of the OAuth flow. Note that you will need an ordinary LinkedIn account in order to login, create an app, and get these credentials. You can create an app through the "Developer" section of your account settings as shown below or by navigating directly to https://www.linkedin.com/secure/developer.

You must ensure that your browser is not blocking popups in order for this script to work.

<img src="files/resources/ch03-linkedin/images/LinkedIn-app.png" width="600px">

## Example 3. Using LinkedIn OAuth credentials to receive an access token an authorize an application

Note: You must ensure that your browser is not blocking popups in order for this script to work. LinkedIn's OAuth flow appears to expressly involve opening a new window, and it does not appear that an inline frame can be used as is the case with some other social web properties. You may also find it very convenient to ensure that you are logged into LinkedIn at http://www.linkedin.com/ with this browser before executing this script, because the OAuth flow will prompt you every time you run it if you are not already logged in. If for some reason you cause IPython Notebook to hang, just select "Kernel => Interrupt" from its menu.

In [ ]:
import os
from threading import Timer
from flask import Flask, request
from linkedin import linkedin
from IPython.display import display
from IPython.display import Javascript as JS

# XXX: Get these values from your application's settings for the OAuth flow

CONSUMER_KEY = ''
CONSUMER_SECRET = ''

# This value is where LinkedIn will redirect. We'll configure an embedded
# web server to be serving requests here

RETURN_URL = 'http://localhost:5000/oauth_helper'

# A temporary file to store a code from the web server
OAUTH_FILE = 'resources/ch03-linkedin/linkedin.authorization_code'

# Configure an emedded web server that accepts one request, stores a file
# that will need to be accessed outside of the request context, and 
# immediately shuts itself down

webserver = Flask("OAuthHelper")
@webserver.route("/oauth_helper")
def oauth_helper():
    code = request.args.get('code')
    f = open(OAUTH_FILE, 'w') # Store the code as a file
    f.write(code)
    f.close()
    shutdown_after_request = request.environ.get('werkzeug.server.shutdown')
    shutdown_after_request()
    return """<p>Handled redirect and extracted code <strong>%s</strong> 
                 for authorization</p>""" % (code,)

# Send an OAuth request to LinkedIn, handle the redirect, and display the access
# token that's included in the redirect for the user to copy and paste

auth = linkedin.LinkedInAuthentication(CONSUMER_KEY, CONSUMER_SECRET, RETURN_URL, 
                                       linkedin.PERMISSIONS.enums.values())

# Display popup after a brief delay to ensure that the web server is running and 
# can handle the redirect back from LinkedIn

Timer(1, lambda: display(JS("window.open('%s')" % auth.authorization_url))).start()

# Run the server to accept the redirect back from LinkedIn and capture the access
# token. This command blocks, but the web server is configured to shut itself down
# after it serves a request, so after the redirect occurs, program flow will continue

webserver.run(host='0.0.0.0')

# As soon as the request finishes, the web server shuts down and these remaining commands
# are executed, which exchange an authorization code for an access token. This process
# seems to need full automation because the authorization code expires very quickly.

auth.authorization_code = open(OAUTH_FILE).read()
auth.get_access_token()

# Prevent stale tokens from sticking around, which could complicate debugging
os.remove(OAUTH_FILE)


# How you can use the application to access the LinkedIn API...
app = linkedin.LinkedInApplication(auth)
print(app.get_profile())